# Installation de Tensorflow


In [22]:
# Installer tensorflow 2.0
# Après l'installation redémarrer l'environnement d'exécution
# en faisant attention à ne pas réexécuter cette cellule.
!pip install --upgrade tensorflow
## cliquer sur Restart Runtime pour utiliser plus tard tensorflow, pour que ça fonctionne

     |████████████████████████████████| 421.8MB 36kB/s 
     |████████████████████████████████| 450kB 49.1MB/s 
     |████████████████████████████████| 3.8MB 49.2MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstal

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
print(tf.__version__)

2.1.0


## Apprentissage sous titres

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Pour les données de validation : 
downloaded = drive.CreateFile({'id': '1eLnvXsKV4Pb2h5oqLAp3fPe42JGxB52B'})
downloaded.GetContentFile('OpenSubtitles.txt')

In [0]:
sous_titres = []
with open("OpenSubtitles.txt", "r", encoding='utf-8') as f:
    for line in f.readlines():
        sous_titres.append(line.strip())

### Aperçu du corpus

In [5]:
sous_titres[:5]

['La Dre Petersen est prêt à vous recevoir.',
 "Désolée. ll faut que j'y aille.",
 "J'avais un jeu parfait. Je vous aurais écrasées.",
 '- Harry va vous conduire.',
 'Surveillez-la attentivement.']

In [6]:
len(sous_titres)

276210

On conserve les 9000 premiers sous titres afin de réduire le temps de calcul.

In [0]:
sous_titres = sous_titres[:9000]

On concatène en une seule chaine de caractère toutes les phrases présentes dans 'sous_titres'.

In [0]:
concat_sous_titres = sous_titres[0]
for phrase in sous_titres[1:] :
  concat_sous_titres = '\n'.join([concat_sous_titres, phrase])


In [9]:
concat_sous_titres

'La Dre Petersen est prêt à vous recevoir.\nDésolée. ll faut que j\'y aille.\nJ\'avais un jeu parfait. Je vous aurais écrasées.\n- Harry va vous conduire.\nSurveillez-la attentivement.\nNe la quittez pas des yeux.\n- Bien. - Vous semblez un peu bilieux.\n- C\'est l\'éclairage.\nJe m\'inquiète pour vous.\nÇa va aller.\nDoit-on aller au bureau de la Dre Petersen ?\nOn ne peut pas aller discuter dans un endroit tranquille ?\nJ\'aimerais bien, si j\'avais le temps.\nVraiment ?\nEntrez.\nVous avez gâché une partie de cartes passionnante, Dre Petersen. Vous pouvez disposer, Harry.\nJe vais attendre dehors.\nJ\'espère que vous allez mieux, Mary. - Pas du tout. - Ça va venir.\nJe trouve tout ça complètement idiot.\nTout ça ?\nLa psychanalyse.\nÇa me barbe complètement.\nS\'allonger sur le divan comme une imbécile, tout raconter.\nVous n\'imaginez pas parvenir à quelque chose en m\'écoutant parler de mon enfance débile.\nMes patients me trouvent toujours prodigieusement agaçante à nos premières

Pré traitement des textes

In [10]:
concat_sous_titres = concat_sous_titres.replace('.','')
concat_sous_titres = concat_sous_titres.replace('\n',' ')
concat_sous_titres = concat_sous_titres.replace('"',' ')
concat_sous_titres

"La Dre Petersen est prêt à vous recevoir Désolée ll faut que j'y aille J'avais un jeu parfait Je vous aurais écrasées - Harry va vous conduire Surveillez-la attentivement Ne la quittez pas des yeux - Bien - Vous semblez un peu bilieux - C'est l'éclairage Je m'inquiète pour vous Ça va aller Doit-on aller au bureau de la Dre Petersen ? On ne peut pas aller discuter dans un endroit tranquille ? J'aimerais bien, si j'avais le temps Vraiment ? Entrez Vous avez gâché une partie de cartes passionnante, Dre Petersen Vous pouvez disposer, Harry Je vais attendre dehors J'espère que vous allez mieux, Mary - Pas du tout - Ça va venir Je trouve tout ça complètement idiot Tout ça ? La psychanalyse Ça me barbe complètement S'allonger sur le divan comme une imbécile, tout raconter Vous n'imaginez pas parvenir à quelque chose en m'écoutant parler de mon enfance débile Mes patients me trouvent toujours prodigieusement agaçante à nos premières rencontres Je vois C'est mon subconscient qui s'énerve Il ne

### Unicité des phrases présentes dans le corpus

On récupère chaque mot dans une liste en les séparant en fonction des espaces présents. On garde les éléments de manière unique, en les triant par ordre alphabétique.

In [11]:
vocab = sorted(np.unique(concat_sous_titres.split(' ')))
print ('{} unique characters'.format(len(vocab)))

9244 unique characters


### Sous titres en représentation numérique

On associé chaque mot des sous titres à un entier différent.

In [12]:
liste_concat_sous_titres = concat_sous_titres.split(' ')
liste_concat_sous_titres[0:4]

['La', 'Dre', 'Petersen', 'est']

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in liste_concat_sous_titres])

In [14]:
text_as_int

array([1086,  620, 1393, ..., 6705, 3662, 5273])

In [15]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ''  :   0,
  '!' :   1,
  '$' :   2,
  '$,':   3,
  "'" :   4,
  "'a":   5,
  "'ont":   6,
  ',' :   7,
  '-' :   8,
  '-500$':   9,
  '-500$?':  10,
  '00':  11,
  '000':  12,
  '05':  13,
  '1' :  14,
  '1,02':  15,
  '1,50':  16,
  '1,65':  17,
  '1,85':  18,
  '1-L-19':  19,
  ...
}


### Affichez la représentation numérique des 13 premières phrases


In [16]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(concat_sous_titres[:13]), text_as_int[:13]))

'La Dre Peters' ---- characters mapped to int ---- > [1086  620 1393 4313 7105 9066 8973 7341  644 5601 4447 7220 5166]


## Partie Prédictions de réponses
Si on passe une phrase comme question, quelle sera la réponse la plus probable ? Entrainement du modèle pour prédire la réponse qui suit.

In [43]:
# Longueur de la phrase maximale qu'on pourra obtenir avec un certain nombre de mots passés en argument
seq_length = 100
examples_per_epoch = len(liste_concat_sous_titres)//(seq_length+1)

# Création d'exemples d'entrainements
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

La
Dre
Petersen
est
prêt


## Création de batchs

La méthode par lots nous permet de convertir facilement ces caractères individuels en séquences de la taille souhaitée

In [44]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(' '.join(idx2char[item.numpy()])))

"La Dre Petersen est prêt à vous recevoir Désolée ll faut que j'y aille J'avais un jeu parfait Je vous aurais écrasées - Harry va vous conduire Surveillez-la attentivement Ne la quittez pas des yeux - Bien - Vous semblez un peu bilieux - C'est l'éclairage Je m'inquiète pour vous Ça va aller Doit-on aller au bureau de la Dre Petersen ? On ne peut pas aller discuter dans un endroit tranquille ? J'aimerais bien, si j'avais le temps Vraiment ? Entrez Vous avez gâché une partie de cartes passionnante, Dre Petersen Vous pouvez disposer, Harry Je vais attendre dehors J'espère"
"que vous allez mieux, Mary - Pas du tout - Ça va venir Je trouve tout ça complètement idiot Tout ça ? La psychanalyse Ça me barbe complètement S'allonger sur le divan comme une imbécile, tout raconter Vous n'imaginez pas parvenir à quelque chose en m'écoutant parler de mon enfance débile Mes patients me trouvent toujours prodigieusement agaçante à nos premières rencontres Je vois C'est mon subconscient qui s'énerve Il 

Appliquer une fonction à tous les batchs

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [46]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(' '.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(' '.join(idx2char[target_example.numpy()])))

Input data:  "La Dre Petersen est prêt à vous recevoir Désolée ll faut que j'y aille J'avais un jeu parfait Je vous aurais écrasées - Harry va vous conduire Surveillez-la attentivement Ne la quittez pas des yeux - Bien - Vous semblez un peu bilieux - C'est l'éclairage Je m'inquiète pour vous Ça va aller Doit-on aller au bureau de la Dre Petersen ? On ne peut pas aller discuter dans un endroit tranquille ? J'aimerais bien, si j'avais le temps Vraiment ? Entrez Vous avez gâché une partie de cartes passionnante, Dre Petersen Vous pouvez disposer, Harry Je vais attendre dehors"
Target data: "Dre Petersen est prêt à vous recevoir Désolée ll faut que j'y aille J'avais un jeu parfait Je vous aurais écrasées - Harry va vous conduire Surveillez-la attentivement Ne la quittez pas des yeux - Bien - Vous semblez un peu bilieux - C'est l'éclairage Je m'inquiète pour vous Ça va aller Doit-on aller au bureau de la Dre Petersen ? On ne peut pas aller discuter dans un endroit tranquille ? J'aimerais bi

In [47]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1086 ('La')
  expected output: 620 ('Dre')
Step    1
  input: 620 ('Dre')
  expected output: 1393 ('Petersen')
Step    2
  input: 1393 ('Petersen')
  expected output: 4313 ('est')
Step    3
  input: 4313 ('est')
  expected output: 7105 ('prêt')
Step    4
  input: 7105 ('prêt')
  expected output: 9066 ('à')


## Création de lots d'entrainements
We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [48]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Construction du modèle

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

## Test du modèle

In [52]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 9244) # (batch_size, sequence_length, vocab_size)


In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           2366464   
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 9244)          9475100   
Total params: 15,779,868
Trainable params: 15,779,868
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [54]:
sampled_indices

array([5153, 4496, 6258, 5776, 9034, 7581, 2442, 5839, 8989, 2037,  362,
       8412, 3117, 2839, 6037, 2271, 7087, 8661, 3632, 8744, 4623, 5963,
       8887, 8644, 2996, 8871, 3835,  196, 1339, 6073, 2616, 4697, 8313,
       8427, 6587,  845, 5122, 2864, 5382, 8315, 7267, 4827, 1570, 7467,
       4165, 6386, 6115, 7581, 8847, 2226, 3567, 4551, 1574, 8521, 3807,
       7897, 8677,  556, 7444, 3012, 5608, 4099, 6572,  853, 3732, 4344,
       1286, 1150, 8108, 8161, 1717, 7171, 2643, 5250, 8889, 7361, 6236,
       8816, 8892, 8341, 6205, 1586, 4725, 4939, 3547, 8733, 7646, 5576,
       8591, 4190, 6553, 9216,  170, 6953, 6770, 2899, 6936, 8147, 6457,
       1442])

In [55]:
print("Input: \n", repr(" ".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2char[sampled_indices ])))

Input: 
 "centre de vacances  en compagnie de son soi-disant patient  - Vous souvenez-vous de cela ? - Non Pourquoi croyez-vous que vous étiez avec lui ? Car où que nous soyons allés, j'en suis revenu avec son identité Je ne serais pas revenu en Edwardes si je n'étais certain de sa mort Comment l'aurais-je su si je n'avais été avec lui à sa mort ? L'étiez-vous ? Je ne me souviens pas mais par déduction, je sais que j'ai dû y être Et toujours par déduction, je sais pourquoi on n'a pas trouvé le corps Parce que je l'ai"

Next Char Predictions: 
 "j'avais feux nazis majeure, zones reverrai aéroports, marché voyez-vous, agents CE tenir complexe, certain monsieur arriverait présent tuant d'une vague frappé miel visiter trouvé, ciel vingt-et-un distinguer Antiquités PRIVE mortel bouffe gardes t'aime, terminé passées Herr irait chambre l'aéroport t'amener raconté général, Rendez-vous rends-moi embrouille ont muté, reverrai vieille approchez d'examiner foin Rentre tout discute savez? tuées Der

# Train the model 

In [56]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 9244)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.131714


In [0]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=100

In [60]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 8 steps
Epoch 1/100
8/8 [==============================] - 3s 318ms/step - loss: 8.6629
Epoch 2/100
8/8 [==============================] - 4s 487ms/step - loss: 7.4549
Epoch 3/100
8/8 [==============================] - 5s 658ms/step - loss: 7.2734
Epoch 4/100
8/8 [==============================] - 1s 160ms/step - loss: 7.2164
Epoch 5/100
8/8 [==============================] - 5s 649ms/step - loss: 7.1881
Epoch 6/100
8/8 [==============================] - 3s 338ms/step - loss: 7.1647
Epoch 7/100
8/8 [==============================] - 1s 158ms/step - loss: 7.1459
Epoch 8/100
8/8 [==============================] - 5s 648ms/step - loss: 7.1177
Epoch 9/100
8/8 [==============================] - 2s 203ms/step - loss: 7.0664
Epoch 10/100
8/8 [==============================] - 4s 561ms/step - loss: 6.9638
Epoch 11/100
8/8 [==============================] - 5s 649ms/step - loss: 6.8263
Epoch 12/100
8/8 [==============================] - 1s 161ms/step - loss: 6.6728
Epoch 13/100
8/8 [=

# Generate text

In [61]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [63]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            2366464   
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 9244)           9475100   
Total params: 15,779,868
Trainable params: 15,779,868
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 20 

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in [start_string] ]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ' '.join(text_generated))


In [74]:
phrase_rep = generate_text(model, start_string="")
phrase_rep=phrase_rep.capitalize()
print(phrase_rep)

Dépression nerveuse rassurez-vous, classe rock rapports suppléants effort, silence, infos habit paniqué, traine blessé mark minutes militaire n'est-ce baisers mark


In [66]:
len(phrase_rep.split())

20